In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as tudata
import torch.optim as optim
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile, common_texts
import nltk
import glob

C:\Users\22251\AppData\Local\conda\conda\envs\my_root\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import torchtext

In [10]:
EPOCH = 10
BATCH_SIZE = 2

In [ ]:
# 读取文件至字符串形式： glob、with+open、
negnames = glob.glob('./train/neg/*.txt')
posnames = glob.glob('./train/pos/*.txt')
negdocuments = []
posdocuments = []
for negname in negnames:
    with open(negname, encoding='utf-8') as negdocument:
        negdocuments.append(negdocument.read())
for posname in posnames:
    with open(posname, encoding='utf-8') as posdocument:
        posdocuments.append(posdocument.read())
        
# 分词： torchtext
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = Field(sequential=False, use_vocab=False)
negwords_list = []
for negdocument in negdocuments:
    negwords_list.append(nltk.word_tokenize(negdocument))
poswords_list = []
for posdocument in posdocuments:
    poswords_list.append(nltk.word_tokenize(posdocument)）

In [5]:
# 读取文件至字符串形式： glob、with+open、
negnames = glob.glob('./train/neg/*.txt')
posnames = glob.glob('./train/pos/*.txt')
negdocuments = []
posdocuments = []
for negname in negnames:
    with open(negname, encoding='utf-8') as negdocument:
        negdocuments.append(negdocument.read())
for posname in posnames:
    with open(posname, encoding='utf-8') as posdocument:
        posdocuments.append(posdocument.read())

# 分词： nltk
negwords_list = []
for negdocument in negdocuments:
    negwords_list.append(nltk.word_tokenize(negdocument))
poswords_list = []
for posdocument in posdocuments:
    poswords_list.append(nltk.word_tokenize(posdocument))

# 读取训练好的模型
model = Word2Vec.load("./word2vec/word2vec.model")

dataset and dataloader

In [6]:
class MYDATA():

    def __init__(self, negwords_list, poswords_list):
        self.datas = negwords_list.copy() + poswords_list.copy()
        self.targets = [0] * 12500 + [1] * 12500

    def __getitem__(self, index):
        selfdatascopy = self.datas.copy()
        datas = selfdatascopy[index].copy()
        targets = self.targets[index]
        for i in range(len(datas)):
            datas[i] = model.wv[datas[i]]
        datas = np.array(datas)
        return datas, targets

    def __len__(self):
        return len(self.datas)
    
mydata = MYDATA(negwords_list, poswords_list)
mydataloader = tudata.DataLoader(mydata, batch_size=BATCH_SIZE, shuffle=False)
mydata.__getitem__(1)[0]

array([[ -1.25913680e-01,   1.79458037e-01,  -7.17329001e-03, ...,
         -1.33494541e-01,  -9.82490965e-05,   8.85094851e-02],
       [ -1.24950195e-02,   1.09412216e-01,   9.14006727e-04, ...,
         -9.09336805e-02,  -1.74252670e-02,   4.16474603e-02],
       [ -1.54697537e+00,   2.83797479e+00,   8.66125703e-01, ...,
         -4.01661754e-01,   1.30564427e+00,  -3.31794113e-01],
       ..., 
       [ -7.30954781e-02,   5.71643949e-01,  -2.05878556e-01, ...,
         -2.38660693e-01,  -1.42768592e-01,  -1.62369773e-01],
       [  1.31036901e+00,   7.19573975e-01,  -2.61321259e+00, ...,
         -2.03342509e+00,   2.13994455e+00,  -1.30881691e+00],
       [ -1.06537521e+00,   1.52517354e+00,   1.10444605e+00, ...,
          7.82284558e-01,   4.93436307e-01,  -2.31320620e+00]], dtype=float32)

network

In [11]:
class ViewClassifier(nn.Module):

    def __init__(self, embedding_dim=100, hidden_dim=100,
                 target_size=2, layer_num=1):
        super(ViewClassifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.layer_num = layer_num
        self.target_size = target_size
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim,
                            self.layer_num, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, 1)

    def forward(self, inputs):
        #sentence, (hn, cn) = inputs
        #words_embedding = nn.Embedding(len(sentence), self.embedding_dim)
        #print(sentence)
        #x = words_embedding(sentence)
        x, (hn, cn) = inputs
        print(x)
        x = x.view(BATCH_SIZE, -1, self.embedding_dim) # batch, seq, embedding_dim
        x, (hn, cn) = self.lstm(x, (hn, cn))
        x = x.view(-1,self.embedding_dim)[-1]
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x, (hn, cn)
    
myclassifier = ViewClassifier()

loss and optimizer

In [8]:
criterion = nn.MSELoss()
optimizer = optim.Adam(myclassifier.parameters(), lr=0.001)

train

In [12]:
for epoch in range(EPOCH):
    running_loss = 0
    hn, cn = torch.zeros(1, BATCH_SIZE, 100), torch.zeros(1, BATCH_SIZE, 100)
    for data in mydataloader:
        inputs, label = data.copy()
        label = label.float()
        optimizer.zero_grad()
        output, (hn, cn) = myclassifier((inputs, (hn, cn)))
        loss = criterion(output, label)
        optimizer.step()
        running_loss += loss.item()
        print(1)
        if i % 500 == 499:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

tensor([[[ 2.9866e-01,  1.8189e+00, -4.4124e-01,  ..., -1.3650e+00,
          -1.7115e+00, -5.9644e-01],
         [ 5.4138e-01,  1.1365e+00, -1.2179e+00,  ...,  7.6875e-01,
          -6.0224e-01, -1.1792e+00],
         [-6.9988e-01,  1.5190e+00, -6.4701e-01,  ..., -2.0301e+00,
           3.2821e+00, -3.6663e-01],
         ...,
         [ 3.2543e+00,  2.5006e+00, -2.1326e+00,  ..., -5.4781e-01,
           6.8150e-01, -5.9686e+00],
         [ 2.6943e-01,  3.3034e-01,  4.4570e-01,  ..., -3.0077e-01,
           1.0101e-01,  3.7009e-01],
         [-1.0654e+00,  1.5252e+00,  1.1044e+00,  ...,  7.8228e-01,
           4.9344e-01, -2.3132e+00]]])


RuntimeError: invalid argument 2: size '[2 x -1 x 100]' is invalid for input with 12300 elements at ..\aten\src\TH\THStorage.cpp:80

In [9]:
for data in mydataloader:
    print(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyError: "word ''n' not in vocabulary"

In [13]:
import pandas